# **Data Gathering**

**Stu. Name**: Amirali Khatib, Amirali Bagherzadeh

**Stu. Id**: 4032315014, 4031315278

**Prof. Name**: Dr. Alireza Shadman

**import & install required libraries**

In [ ]:
! pip install mplsoccer
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch, VerticalPitch
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 3.9 MB/s eta 0:00:00


# **DATA EXTRACTION FOR XPASS**

In [ ]:
def get_competition():
  comp = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/competitions.json').json()
  comp = pd.DataFrame({
        'competition_id': [comp[i]['competition_id'] for i in range(len(comp))],
        'competition_name': [comp[i]['competition_name'] for i in range(len(comp))],
        'season_name': [comp[i]['season_name'] for i in range(len(comp))],
        'season_id': [comp[i]['season_id'] for i in range(len(comp))]
    })
  return comp

In [ ]:
comp = get_competition()
comp

,competition_id,competition_name,season_name,season_id
0,9,1. Bundesliga,2023/2024,281
1,9,1. Bundesliga,2015/2016,27
2,1267,African Cup of Nations,2023,107
3,16,Champions League,2018/2019,4
4,16,Champions League,2017/2018,1
...,...,...,...,...
69,55,UEFA Euro,2020,43
70,35,UEFA Europa League,1988/1989,75
71,53,UEFA Women's Euro,2022,106
72,72,Women's World Cup,2023,107


In [ ]:
def get_matches():
  MATCHES = pd.DataFrame()
  for c_id, s_id in zip(comp.competition_id, comp.season_id):
    matches = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/'+ str(c_id)+ '/' + str(s_id) + '.json').json()
    matches = pd.DataFrame({
          'match_id':  [matches[i]['match_id'] for i in range(len(matches))],
          'match_date':[matches[i]['match_date'] for i in range(len(matches))],
          'home_team': [matches[i]['home_team']['home_team_name'] for i in range(len(matches))],
          'away_team': [matches[i]['away_team']['away_team_name'] for i in range(len(matches))],
          'h_score': [matches[i]['home_score'] for i in range(len(matches))],
          'a_score': [matches[i]['away_score'] for i in range(len(matches))]})
    MATCHES = pd.concat([MATCHES, matches])
  MATCHES.index = list(range(len(MATCHES)))
  return MATCHES

In [ ]:
MATCHES = get_matches()
MATCHES

,match_id,match_date,home_team,away_team,h_score,a_score
0,3895302,2024-04-14,Bayer Leverkusen,Werder Bremen,5,0
1,3895292,2024-04-06,Union Berlin,Bayer Leverkusen,0,1
2,3895333,2024-05-05,Eintracht Frankfurt,Bayer Leverkusen,1,5
3,3895340,2024-05-12,Bochum,Bayer Leverkusen,0,5
4,3895348,2024-05-18,Bayer Leverkusen,Augsburg,2,1
...,...,...,...,...,...,...
3428,69199,2019-06-27,Norway Women's,England Women's,0,3
3429,69258,2019-07-02,England Women's,United States Women's,1,2
3430,69284,2019-07-03,Netherlands Women's,Sweden Women's,1,0
3431,69205,2019-06-29,Italy Women's,Netherlands Women's,0,2


In [ ]:
def isin_360(match_id):
  response = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/refs/heads/master/data/three-sixty/'+ str(match_id) +'.json')
  if response.status_code == 404:
    return False
  elif response.status_code == 200:
    return True
  else:
    return 'Eliminated'

In [ ]:
MATCHES['isin360'] = [isin_360(MATCHES.loc[i]['match_id']) for i in range(len(MATCHES))]
MATCHES

In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/SL-Project/DATASETS')

In [ ]:
MATCHES.to_pickle('/content/drive/MyDrive/SL-Project/DATASETS/MATCHES.pkl')

In [ ]:
! ls

MATCHES.pkl


In [ ]:
MATCHES = pd.read_pickle('/content/drive/MyDrive/SL-Project/DATASETS/MATCHES.pkl')


In [ ]:
MATCHES = MATCHES[MATCHES.isin360 == True]
MATCHES.index = list(range(len(MATCHES)))
MATCHES

,match_id,match_date,home_team,away_team,h_score,a_score,isin360
0,3895302,2024-04-14,Bayer Leverkusen,Werder Bremen,5,0,True
1,3895292,2024-04-06,Union Berlin,Bayer Leverkusen,0,1,True
2,3895333,2024-05-05,Eintracht Frankfurt,Bayer Leverkusen,1,5,True
3,3895340,2024-05-12,Bochum,Bayer Leverkusen,0,5,True
4,3895348,2024-05-18,Bayer Leverkusen,Augsburg,2,1,True
...,...,...,...,...,...,...,...
290,3893800,2023-07-24,Italy Women's,Argentina Women's,1,0,True
291,3893799,2023-07-24,Germany Women's,Morocco Women's,6,0,True
292,3893796,2023-07-23,Sweden Women's,South Africa Women's,2,1,True
293,3893795,2023-07-22,Denmark Women's,China PR Women's,1,0,True


In [ ]:
data = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/'+ str(7584) +'.json').json()
data = list(filter(lambda x: x['type']['name'] == 'Pass', data))
for i in range(len(data)):
  try:
    u_p = data[i]['under_pressure']
    print(u_p)
  except: continue

In [ ]:
def pass_event_detail(events, i):
  # ...
  try: s_loc = events[i]['location']
  except: s_loc = None
  # ...
  try: length = events[i]['pass']['length']
  except: length = None
  # ...
  try: angle = events[i]['pass']['angle']
  except: angle = None
  # ...
  try: height = events[i]['pass']['height']
  except: height = None
  # ...
  try: e_loc = events[i]['pass']['end_location']
  except: e_loc = None
  # ...
  try: b_part = events[i]['pass']['body_part']['name']
  except: b_part = None
  # ...
  try: deflected = events[i]['pass']['deflected']
  except: deflected = False
  # ...
  try: backheel = events[i]['pass']['backheel']
  except: backheel = False
  # ...
  try: outcome = events[i]['pass']['outcome']['name']
  except: outcome = 'complete'
  # ...
  try: cross = events[i]['pass']['cross']
  except: cross = False
  # ...
  try: cut_back = events[i]['pass']['cut_back']
  except: cut_back = False
  # ...
  try: switch = events[i]['pass']['switch']
  except: switch = False
  # ...
  try: under_pressure = events[i]['under_pressure']
  except: under_pressure = False

  return {
      'start_loc': s_loc,
      'length': length,
      'angle': angle,
      'height': height,
      'end_loc': e_loc,
      'b_part': b_part,
      'deflected': deflected,
      'backheel': backheel,
      'cut_back': cut_back,
      'cross': cross,
      'switch': switch,
      'under_pressure': under_pressure,
      'outcome': outcome
  }

In [ ]:
def get_xpass_events(m_ids):
  Xpass = pd.DataFrame()
  for i, id in list(enumerate(m_ids)):
    if i % 100 == 0:
      print(f'Match #{i}: Added')
    events = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/'+ str(id) +'.json').json()
    events = list(filter(lambda x: x['type']['name'] == 'Pass', events))
    events = pd.DataFrame(
        {
         'match_id': [id for i in range(len(events))],
         'event_id': [events[i]['id'] for i in range(len(events))],
         'event_time': [(events[i]['minute'], events[i]['second']) for i in range(len(events))],
         'event_period': [events[i]['period'] for i in range(len(events))],
         'play_pattern': [events[i]['play_pattern']['name'] for i in range(len(events))],
         'team': [events[i]['team']['name'] for i in range(len(events))],
         'event_name': [events[i]['type']['name'] for i in range(len(events))],
         'event_details': [pass_event_detail(events, i) for i in range(len(events))]
        }
    )
    Xpass = pd.concat([Xpass, events])

    if i % 100 == 0:
      print(f'Length: {len(Xpass)}')

  return Xpass
# print(list(MATCHES.match_id))
xpass = get_xpass_events(list(MATCHES.match_id))
xpass

In [ ]:
xpass.index = list(range(len(xpass)))
xpass.tail()

,match_id,event_id,event_time,event_period,play_pattern,team,event_name,event_details
305523,3893793,9582979a-7ba1-44f6-9bf5-a8291815e57a,"(96, 24)",2,Regular Play,Japan Women's,Pass,"{'start_loc': [50.3, 78.2], 'length': 7.77946,..."
305524,3893793,f249814e-1547-44c0-8db4-9932eca0e714,"(96, 25)",2,Regular Play,Japan Women's,Pass,"{'start_loc': [57.7, 75.8], 'length': 7.026378..."
305525,3893793,5247be79-6ae6-40b0-a5cf-93cc51164fe6,"(96, 26)",2,Regular Play,Japan Women's,Pass,"{'start_loc': [64.1, 78.7], 'length': 10.24499..."
305526,3893793,2483c640-347f-4ccb-9415-6ad71ad62653,"(96, 27)",2,Regular Play,Japan Women's,Pass,"{'start_loc': [72.1, 72.3], 'length': 14.16509..."
305527,3893793,02eace19-8fef-4b15-a664-4c831cf39824,"(96, 29)",2,Regular Play,Japan Women's,Pass,"{'start_loc': [69.0, 57.1], 'length': 34.1826,..."


In [ ]:
xpass.head()

,match_id,event_id,event_time,event_period,play_pattern,team,event_name,event_details
0,3895302,221b0c8d-6386-4ae8-bb4a-a1dc98742312,"(0, 3)",1,From Kick Off,Werder Bremen,Pass,"{'start_loc': [61.0, 40.1], 'length': 2.154065..."
1,3895302,ff56e821-21e9-4cef-ba2a-7eb5eb3769c6,"(0, 4)",1,From Kick Off,Werder Bremen,Pass,"{'start_loc': [59.0, 40.6], 'length': 24.04100..."
2,3895302,4464cb75-f45f-4508-8444-2560d1625d06,"(0, 7)",1,From Kick Off,Werder Bremen,Pass,"{'start_loc': [35.1, 43.7], 'length': 20.18960..."
3,3895302,b88bbf46-30c8-4c60-b8cc-6d855f94f990,"(0, 10)",1,From Kick Off,Werder Bremen,Pass,"{'start_loc': [32.7, 65.3], 'length': 17.18982..."
4,3895302,63a6070e-0c56-4615-8bab-8f6792d5562a,"(0, 11)",1,From Kick Off,Werder Bremen,Pass,"{'start_loc': [44.5, 77.5], 'length': 21.21744..."


In [ ]:
xpass[['match_id','event_id']]

,match_id,event_id
0,3895302,221b0c8d-6386-4ae8-bb4a-a1dc98742312
1,3895302,ff56e821-21e9-4cef-ba2a-7eb5eb3769c6
2,3895302,4464cb75-f45f-4508-8444-2560d1625d06
3,3895302,b88bbf46-30c8-4c60-b8cc-6d855f94f990
4,3895302,63a6070e-0c56-4615-8bab-8f6792d5562a
...,...,...
305523,3893793,9582979a-7ba1-44f6-9bf5-a8291815e57a
305524,3893793,f249814e-1547-44c0-8db4-9932eca0e714
305525,3893793,5247be79-6ae6-40b0-a5cf-93cc51164fe6
305526,3893793,2483c640-347f-4ccb-9415-6ad71ad62653


In [ ]:
def get_360(m_ids, passes):
  dict_360 = {}
  dict_360['event_uuid'] = []
  dict_360['freeze_frame'] = []

  for i, id in list(enumerate(m_ids)):

    try:
      event_360 = requests.get('https://github.com/statsbomb/open-data/raw/refs/heads/master/data/three-sixty/'+ str(id)+ '.json').json()
      print(f'{i}-{id}')
      temp = passes[passes.match_id == id]
      for i in range(len(event_360)):
        if event_360[i]['event_uuid'] in list(temp.event_id):
          dict_360['event_uuid'].append(event_360[i]['event_uuid'])
          dict_360['freeze_frame'].append(event_360[i]['freeze_frame'])
        else: continue

    except: continue

  return pd.DataFrame(dict_360)

In [ ]:
detail_360 = get_360(list(MATCHES.match_id), xpass[['match_id','event_id']])

In [ ]:
detail_360.to_pickle('/content/drive/MyDrive/SL-Project/DATASETS/detail_360.pkl')

In [ ]:
xpass.to_pickle('/content/drive/MyDrive/SL-Project/DATASETS/xpass_events.pkl')

In [ ]:
len(xpass), len(detail_360)

In [ ]:
requests.get('https://github.com/statsbomb/open-data/raw/refs/heads/master/data/three-sixty/'+ str(3788741)+ '.json').json()

# **DATA EXTRACTION FOR XG**

In [ ]:
! pip install mplsoccer
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch, VerticalPitch
import json

In [ ]:
def get_competition():
  comp = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/competitions.json').json()
  comp = pd.DataFrame({
        'competition_id': [comp[i]['competition_id'] for i in range(len(comp))],
        'competition_name': [comp[i]['competition_name'] for i in range(len(comp))],
        'season_name': [comp[i]['season_name'] for i in range(len(comp))],
        'season_id': [comp[i]['season_id'] for i in range(len(comp))]
    })
  return comp

In [ ]:
def get_matches(comp):
  MATCHES = pd.DataFrame()
  for c_id, s_id in zip(comp.competition_id, comp.season_id):
    matches = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/'+ str(c_id)+ '/' + str(s_id) + '.json').json()
    matches = pd.DataFrame({
          'match_id':  [matches[i]['match_id'] for i in range(len(matches))],
          'match_date':[matches[i]['match_date'] for i in range(len(matches))],
          'home_team': [matches[i]['home_team']['home_team_name'] for i in range(len(matches))],
          'away_team': [matches[i]['away_team']['away_team_name'] for i in range(len(matches))],
          'h_score': [matches[i]['home_score'] for i in range(len(matches))],
          'a_score': [matches[i]['away_score'] for i in range(len(matches))]})
    MATCHES = pd.concat([MATCHES, matches])
  MATCHES.index = list(range(len(MATCHES)))
  return MATCHES

In [ ]:
def shot_event_detail(events, i):
  # ...
  try: start_loc = events[i]['location']
  except: start_loc = "None"
  # ...
  try: under_pressure = events[i]['under_pressure']
  except: under_pressure = False
  # ...
  try: follows_dribble = events[i]['shot']['follows_dribble']
  except: follows_dribble = False
  # ...
  try: first_time = events[i]['shot']['first_time']
  except: first_time = False
  # ...
  try: technique = events[i]['shot']['technique']['name']
  except: technique = "None"
  # ...
  try: body_part = events[i]['shot']['body_part']['name']
  except: body_part = "None"
  # ...
  try: open_goal = events[i]['shot']['open_goal']
  except: open_goal = False
  # ...
  try: xg = events[i]['shot']['statsbomb_xg']
  except: xg = "None"
  # ...
  try: shot_type = events[i]['shot']['type']['name']
  except: shot_type = "None"
  # ...
  try: shot_outcome = events[i]['shot']['outcome']['name']
  except: shot_outcome = "None"
  # ...
  try: end_loc = events[i]['shot']['end_location']
  except: end_loc = "None"
  # ...
  try:
    len_frame_360 = len(events[i]['shot']['freeze_frame'])
    frame_360 = [[events[i]['shot']['freeze_frame'][j]['location'],events[i]['shot']['freeze_frame'][j]['teammate']] for j in range(len_frame_360)]
  except: frame_360 = "None"
  # ...
  try: deflected = events[i]['shot']['deflected']
  except: deflected = "None"


  return {
      'start_loc': start_loc,
      'end_loc': end_loc,
      'under_pressure': under_pressure,
      'follows_dribble': follows_dribble,
      'first_time': first_time,
      'technique': technique,
      'body_part': body_part,
      'open_goal': open_goal,
      'shot_type': shot_type,
      'shot_outcome': shot_outcome,
      'deflected': deflected,
      'freeze_frame': frame_360,
      'xg': xg,
  }

In [ ]:
len_frame_360 = len(events[0]['shot']['freeze_frame'])
frame_360 = [[events[0]['shot']['freeze_frame'][j]['location'],events[0]['shot']['freeze_frame'][j]['teammate']] for j in range(len_frame_360)]
frame_360

In [ ]:
events = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/'+ str(3895292) +'.json').json()
events = list(filter(lambda x: x['type']['name'] == 'Shot', events))

In [ ]:
1 in [1,2,3]

In [ ]:
sample = events[0]['shot']['freeze_frame']
# list(filter(lambda x: ('location' in x.keys())&('teammate' in x.keys()) ,sample))
list(filter(lambda x: ('location' in list(x.keys())) ,sample))

In [ ]:
def get_shot_events(m_ids):
  Shot = pd.DataFrame()
  for i, id in list(enumerate(m_ids)):
    if i % 100 == 0:
      print(f'Match #{i}: Added')
    events = requests.get('https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/'+ str(id) +'.json').json()
    events = list(filter(lambda x: x['type']['name'] == 'Shot', events))
    events_df = pd.DataFrame(
        {
         'match_id': [id for i in range(len(events))],
         'event_id': [events[i]['id'] for i in range(len(events))],
         'event_time': [(events[i]['minute'], events[i]['second']) for i in range(len(events))],
         'event_period': [events[i]['period'] for i in range(len(events))],
         'play_pattern': [events[i]['play_pattern']['name'] for i in range(len(events))],
         'team': [events[i]['team']['name'] for i in range(len(events))],
         'event_name': [events[i]['type']['name'] for i in range(len(events))],
         'event_details': [shot_event_detail(events, i) for i in range(len(events))],
        }
    )
    Shot = pd.concat([Shot, events_df])
    if i % 100 == 0:
      print(f'Length: {len(Shot)}')
  return Shot

In [ ]:
COMP = get_competition()
MATCHES = get_matches(COMP)
SHOT = get_shot_events(list(MATCHES.match_id))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
SHOT.index = list(range(len(SHOT)))
SHOT.to_pickle('/content/drive/MyDrive/SL-Project/DATASETS/XG/SHOT_EVENT_V01.pkl')#,SHOT)

In [ ]:
SHOT.loc[0]['event_details']

In [ ]:
sample = SHOT.loc[0]['event_details']['freeze_frame']
list(filter(lambda x: x[1] == True ,sample))